# Base de dados: Absenteeism at work
###Fonte: https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

In [ ]:
!pip install -U pandas-profiling

     |████████████████████████████████| 261 kB 5.1 MB/s 
     |████████████████████████████████| 10.9 MB 57.2 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
     |████████████████████████████████| 303 kB 13.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 675 kB 59.8 MB/s 
     |████████████████████████████████| 102 kB 7.6 MB/s 
     |████████████████████████████████| 3.1 MB 39.3 MB/s 
     |████████████████████████████████| 812 kB 38.7 MB/s 
     |████████████████████████████████| 38.1 MB 1.8 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=4843c4c9f7d1f28a702ca9a96d7ffb38cc0df1d1ac1e3ff25a23f37309ad1370
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=14e0da26490c792f430f5f54e7967921966a371d3aa17288dc2f71d379e2a5e7
  

In [ ]:
%pip install -U -q dtreeviz seaborn yellowbrick

     |████████████████████████████████| 61 kB 457 bytes/s 


In [ ]:
!pip install scipy 

In [ ]:
# importar numpy para trabalhar com matrizes e pandas para ler dados no formato frame
import numpy as np
import pandas as pd

# importar o pacote datasets do scikit-learn para carregar conjunto de dados
from sklearn import datasets

# importar KMeans para fazer agrupamento
from sklearn.cluster import KMeans

# importar o módulo pyplot para plotar o gráfico no método do cotovelo
import matplotlib.pyplot as plt

# Importar a classe MinMaxScaler para fazer pré-processamento
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Importar pandas_profiling para gerar relatório de esploração dos dados
from pandas_profiling import ProfileReport

import sklearn.metrics as sm 

from sklearn.cluster import DBSCAN

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Absenteeism_at_work.csv to Absenteeism_at_work.csv


In [ ]:
import pandas as pd
import io
ds = pd.read_csv(io.StringIO(uploaded['Absenteeism_at_work.csv'].decode('utf-8')),header=0, sep=';')
ds.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,Hit target,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,97,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,97,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,97,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,97,0,1,2,1,0,1,90,172,30,2


# ANÁLISE EXPLORATÓRIA DOS DADOS

In [ ]:
# Informações sobre tipos de variáveis
ds.info()

# abaixo temos os seguintes tipos de variáveis: "int64' e "float64". Entretanto, sabemos que muitas variáveis os números, na verdade, representam categorias
# Exemplos disto: Reason for absence, Month of absence e Day of the week etc. Logo, essas varíavem devem ser tratadas.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               740 non-null    int64  
 1   Reason for absence               740 non-null    int64  
 2   Month of absence                 740 non-null    int64  
 3   Day of the week                  740 non-null    int64  
 4   Seasons                          740 non-null    int64  
 5   Transportation expense           740 non-null    int64  
 6   Distance from Residence to Work  740 non-null    int64  
 7   Service time                     740 non-null    int64  
 8   Age                              740 non-null    int64  
 9   Work load Average/day            740 non-null    float64
 10  Hit target                       740 non-null    int64  
 11  Disciplinary failure             740 non-null    int64  
 12  Education             

In [ ]:
profile = ProfileReport(ds, title='Absenteism',html={'style':{'full_width':True}})

profile.to_notebook_iframe()
# O relatório abaixo indica:
## i. há 26 instâncias duplicadas. Conclusão: não serão removidas, pois aparentemente não representam erros; e
## ii. há diversas correlações fortes entre as variávies. Algumas delas serão removidas: service time, Month of absence e weight.

Output hidden; open in https://colab.research.google.com to view.

# PRÉ-PROCESSAMENTO

In [ ]:
# Removendo atributos: i) sugeridos na etapa anterior, ii) Hit target, por não haver informaçõs suficientes e iii) ID
ds01 = ds.drop(columns=['Service time', 'Month of absence', 'Weight', 'Hit target', 'ID'])
ds01

,Reason for absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Age,Work load Average/day,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Height,Body mass index,Absenteeism time in hours
0,26,3,1,289,36,33,239.554,0,1,2,1,0,1,172,30,4
1,0,3,1,118,13,50,239.554,1,1,1,1,0,0,178,31,0
2,23,4,1,179,51,38,239.554,0,1,0,1,0,0,170,31,2
3,7,5,1,279,5,39,239.554,0,1,2,1,1,0,168,24,4
4,23,5,1,289,36,33,239.554,0,1,2,1,0,1,172,30,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,14,3,1,289,36,33,264.604,0,1,2,1,0,1,172,30,8
736,11,3,1,235,11,37,264.604,0,3,1,0,0,1,172,29,4
737,0,3,1,118,14,40,271.219,0,1,1,1,0,8,170,34,0
738,0,4,2,231,35,39,271.219,0,1,2,1,0,2,170,35,0


In [ ]:
ds01['Reason for absence'].value_counts()

23    149
28    112
27     69
13     55
0      43
19     40
22     38
26     33
25     31
11     26
10     25
18     21
14     19
1      16
7      15
6       8
12      8
21      6
8       6
9       4
5       3
16      3
24      3
15      2
4       2
3       1
2       1
17      1
Name: Reason for absence, dtype: int64

In [ ]:
# Definir a lista de atributos categóricos e atributos contínuos
categorical_features = ['Reason for absence', 'Day of the week', 'Seasons', 'Disciplinary failure', 'Education', 'Social drinker', 'Social smoker', 'Pet']
continuous_features = ['Transportation expense', 'Distance from Residence to Work', 'Age',	'Work load Average/day',
                       'Son', 'Height', 'Body mass index', 'Absenteeism time in hours']

# Transformar atributos categóricos em binários com a função get_dummies()
for col in categorical_features:
  dummies = pd.get_dummies(ds01[col], prefix=col)
  df = pd.concat([ds01, dummies], axis=1)
  df.drop(col, axis=1, inplace=True)
df.head()

,Reason for absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Age,Work load Average/day,Disciplinary failure,Education,Son,Social drinker,Social smoker,Height,Body mass index,Absenteeism time in hours,Pet_0,Pet_1,Pet_2,Pet_4,Pet_5,Pet_8
0,26,3,1,289,36,33,239.554,0,1,2,1,0,172,30,4,0,1,0,0,0,0
1,0,3,1,118,13,50,239.554,1,1,1,1,0,178,31,0,1,0,0,0,0,0
2,23,4,1,179,51,38,239.554,0,1,0,1,0,170,31,2,1,0,0,0,0,0
3,7,5,1,279,5,39,239.554,0,1,2,1,1,168,24,4,1,0,0,0,0,0
4,23,5,1,289,36,33,239.554,0,1,2,1,0,172,30,2,0,1,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Reason for absence               740 non-null    int64  
 1   Day of the week                  740 non-null    int64  
 2   Seasons                          740 non-null    int64  
 3   Transportation expense           740 non-null    int64  
 4   Distance from Residence to Work  740 non-null    int64  
 5   Age                              740 non-null    int64  
 6   Work load Average/day            740 non-null    float64
 7   Disciplinary failure             740 non-null    int64  
 8   Education                        740 non-null    int64  
 9   Son                              740 non-null    int64  
 10  Social drinker                   740 non-null    int64  
 11  Social smoker                    740 non-null    int64  
 12  Height                

In [ ]:
# Normalizar os atributos contínuos com MinMaxScaler()

mms = MinMaxScaler()

mms.fit(df)
df_transformed = mms.transform(df)
df_transformed = pd.DataFrame(df_transformed)
df_transformed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,0.928571,0.25,0.0,0.633333,0.659574,0.193548,0.194471,0.0,0.0,0.50,1.0,0.0,0.272727,0.578947,0.033333,0.0,1.0,0.0,0.0,0.0,0.0
1,0.000000,0.25,0.0,0.000000,0.170213,0.741935,0.194471,1.0,0.0,0.25,1.0,0.0,0.454545,0.631579,0.000000,1.0,0.0,0.0,0.0,0.0,0.0
2,0.821429,0.50,0.0,0.225926,0.978723,0.354839,0.194471,0.0,0.0,0.00,1.0,0.0,0.212121,0.631579,0.016667,1.0,0.0,0.0,0.0,0.0,0.0
3,0.250000,0.75,0.0,0.596296,0.000000,0.387097,0.194471,0.0,0.0,0.50,1.0,1.0,0.151515,0.263158,0.033333,1.0,0.0,0.0,0.0,0.0,0.0
4,0.821429,0.75,0.0,0.633333,0.659574,0.193548,0.194471,0.0,0.0,0.50,1.0,0.0,0.272727,0.578947,0.016667,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# Informações sobre tipos de variáveis
df_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       740 non-null    float64
 1   1       740 non-null    float64
 2   2       740 non-null    float64
 3   3       740 non-null    float64
 4   4       740 non-null    float64
 5   5       740 non-null    float64
 6   6       740 non-null    float64
 7   7       740 non-null    float64
 8   8       740 non-null    float64
 9   9       740 non-null    float64
 10  10      740 non-null    float64
 11  11      740 non-null    float64
 12  12      740 non-null    float64
 13  13      740 non-null    float64
 14  14      740 non-null    float64
 15  15      740 non-null    float64
 16  16      740 non-null    float64
 17  17      740 non-null    float64
 18  18      740 non-null    float64
 19  19      740 non-null    float64
 20  20      740 non-null    float64
dtypes: float64(21)
memory usage: 121.5 KB


In [ ]:
# GERAR O "df_transformed"
df_transformed.to_csv('df_transformed.csv', index=False, encoding='utf-8')